# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,coquimbo,-29.9533,-71.3436,288.00,82,75,5.14,CL,1696876875
1,1,quellon,-43.1167,-73.6167,282.22,90,100,4.51,CL,1696876971
2,2,port-aux-francais,-49.3500,70.2167,275.84,79,70,16.54,TF,1696876972
3,3,port mathurin,-19.6833,63.4167,296.69,74,5,6.48,MU,1696876974
4,4,lebu,-37.6167,-73.6500,285.85,71,100,6.57,CL,1696876856


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    geo=True,  # Enable geographic projection
    tiles="CartoLight",
    label="City Humidity",
    size=city_data_df["Humidity"] / 5,  # Adjust the divisor for better visualization
    title="City Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    colorbar=True,
    cmap="coolwarm",
    width=800,
    height=400,
)

# Display the map
humidity_map

:Overlay
   .Tiles.I              :Tiles   [x,y]
   .Points.City_Humidity :Points   [Lng,Lat]   (Humidity,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 80) & \
                   (city_data_df["Humidity"] < 60) & \
                   (city_data_df["Cloudiness"] < 20)

ideal_cities_df = city_data_df.loc[ideal_conditions]

# Drop any rows with null values
city_data_df.dropna(subset=["Max Temp", "Humidity", "Cloudiness"], inplace=True)


# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,coquimbo,-29.9533,-71.3436,288.00,82,75,5.14,CL,1696876875
1,1,quellon,-43.1167,-73.6167,282.22,90,100,4.51,CL,1696876971
2,2,port-aux-francais,-49.3500,70.2167,275.84,79,70,16.54,TF,1696876972
3,3,port mathurin,-19.6833,63.4167,296.69,74,5,6.48,MU,1696876974
4,4,lebu,-37.6167,-73.6500,285.85,71,100,6.57,CL,1696876856
...,...,...,...,...,...,...,...,...,...,...
545,545,urucara,-2.5364,-57.7600,314.76,18,6,3.00,BR,1696877464
546,546,childress,34.4264,-100.2040,300.33,32,0,0.00,US,1696877464
547,547,takaungu,-3.6837,39.8566,299.42,86,6,5.22,KE,1696877464
548,548,binzhou,37.3667,118.0167,286.71,56,41,1.70,CN,1696877465


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,coquimbo,-29.9533,-71.3436,288.00,82,75,5.14,CL,1696876875,
1,1,quellon,-43.1167,-73.6167,282.22,90,100,4.51,CL,1696876971,
2,2,port-aux-francais,-49.3500,70.2167,275.84,79,70,16.54,TF,1696876972,
3,3,port mathurin,-19.6833,63.4167,296.69,74,5,6.48,MU,1696876974,
4,4,lebu,-37.6167,-73.6500,285.85,71,100,6.57,CL,1696876856,
...,...,...,...,...,...,...,...,...,...,...,...
545,545,urucara,-2.5364,-57.7600,314.76,18,6,3.00,BR,1696877464,
546,546,childress,34.4264,-100.2040,300.33,32,0,0.00,US,1696877464,
547,547,takaungu,-3.6837,39.8566,299.42,86,6,5.22,KE,1696877464,
548,548,binzhou,37.3667,118.0167,286.71,56,41,1.70,CN,1696877465,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters (10 kilometers)
params = {
    "categories": "hotels",
    "limit": 1,  # Limit the results to the first hotel found
    "apiKey": "geoapify_key",  # Replace with your actual Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distance(point, {lat},{lon}) <= {radius}"
    params["bias"] = f"point({lat},{lon})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
coquimbo - nearest hotel: No hotel found
quellon - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
la paz - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
lae - nearest hotel: No hotel found
inyonga - nearest hotel: No hotel found
isla del bosque - nearest hotel: No hotel found
olinda - nearest hotel: No hotel found
fuli - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
chipata - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
okhotsk - nearest hotel: No hotel found
hasaki - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
port hueneme - ne

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,coquimbo,-29.9533,-71.3436,288.00,82,75,5.14,CL,1696876875,No hotel found
1,1,quellon,-43.1167,-73.6167,282.22,90,100,4.51,CL,1696876971,No hotel found
2,2,port-aux-francais,-49.3500,70.2167,275.84,79,70,16.54,TF,1696876972,No hotel found
3,3,port mathurin,-19.6833,63.4167,296.69,74,5,6.48,MU,1696876974,No hotel found
4,4,lebu,-37.6167,-73.6500,285.85,71,100,6.57,CL,1696876856,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
545,545,urucara,-2.5364,-57.7600,314.76,18,6,3.00,BR,1696877464,No hotel found
546,546,childress,34.4264,-100.2040,300.33,32,0,0.00,US,1696877464,No hotel found
547,547,takaungu,-3.6837,39.8566,299.42,86,6,5.22,KE,1696877464,No hotel found
548,548,binzhou,37.3667,118.0167,286.71,56,41,1.70,CN,1696877465,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    geo=True,
    tiles="CartoLight",
    label="City Humidity",
    size=city_data_df["Humidity"] / 5,  # Adjust the divisor for better visualization
    title="City Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    colorbar=True,
    cmap="coolwarm",
    width=800,
    height=400,
    hover_cols=["City", "Country", "Humidity", "Hotel Name"],  # Add "Hotel Name" and "Country" to hover information
)


# Display the map
humidity_map

:Overlay
   .Tiles.I              :Tiles   [x,y]
   .Points.City_Humidity :Points   [Lng,Lat]   (Humidity,_size,City,Country)